# Enhancement of collected data

This notebook performs a domain analysis based on the data `real_domains.csv`. First, it checks the correctness of the specified `A records` and `MX records`. The current A-records and MX-records of the top-level domain are determined. In the next step, the `redirects` and `HTTP status codes` are obtained from the top-level domains. The Python package called `dnspython` is used for the determination. In addition, a further analysis of the MX records determines their locations and the corresponding ASNs. Free IP geolocation databases from MaxMind are used for this.

## Setup

First, the required installations and imports are made, the session is initialized and the basis data is collected from the Postgres database:

In [ ]:
# Required installations
!pip install ipynb
!pip install dnspython
!pip install geoip2

# Required imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# PostgreSQL access data
host = "bda_gr4_database"
port = "5432"
database = "domainanalysis"
user = "postgres"
password = "postgres"

# PostgreSQL connection url
connection = f"jdbc:postgresql://{host}:{port}/{database}"

# Create a Spark session
spark = SparkSession.builder \
    .appName("domain_analysis") \
    .getOrCreate()

# Read data from the database
domains_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", "domain") \
                .option("user", user) \
                .option("password", password) \
                .load()

# Display the data frame
domains_df.limit(15).toPandas()

## 1. Validation of A and MX records

In the following chapter, the functions are imported from the notebook `Functions.ipynb` in order to use them for the definition of the `UDFs`. In doing so, the current `a-records` and `mx-records` are to be determined from the `top-level-domain`.  If the records cannot be determined, the returned errors are stored in separate columns:

In [ ]:
# Import functions from Funtions.ipynb
from ipynb.fs.full.Functions import getARecords, getARecords_error, getMXRecords, getMXRecords_error

# Creating of UDF's
udf_getARecords = udf(getARecords, ArrayType(StringType()))
udf_getARecords_error = udf(getARecords_error, IntegerType())
udf_getMXRecords = udf(getMXRecords, ArrayType(StringType()))
udf_getMXRecords_error = udf(getMXRecords_error, IntegerType())

Next, the A and MX records are checked by calling the UDFs and creating the returned results in new columns:

In [ ]:
# Create the new columns with the results
domains_checked_df = domains_df.withColumn("a_record_checked", udf_getARecords("top_level_domain")) \
                            .withColumn("a_record_checked_error", udf_getARecords_error("top_level_domain")) \
                            .withColumn("mx_record_checked", udf_getMXRecords("top_level_domain")) \
                            .withColumn("mx_record_checked_error", udf_getMXRecords_error("top_level_domain"))

In preparation for writing to the database, the data frame is put into the correct order:

In [ ]:
# Changing the order of the data frame
domains_checked_df = domains_checked_df.select("top_level_domain", "a_record", "a_record_checked", "a_record_checked_error", "mx_record", "mx_record_checked", "mx_record_checked_error")

Last but not least, let's check if the records have either not been changed, changed completely or partially in comparison to the original data.

In [ ]:
# Count the occrence of checked A-records
a_record_checked_count = domains_df.withColumn('a_record_checked', explode(col('a_record'))) \
        .groupBy('a_record_checked') \
        .count()

# Count the occrence of checked MX-records
mx_record_checked_count = domains_df.withColumn('mx_record_checked', explode(col('mx_record'))) \
        .groupBy('mx_record_checked') \
        .count()

# Finally, create new data frames only containing the top 10 A-/MX-records
a_record_count_top_ten_df = a_record_checked_count.orderBy(['count'], ascending = [False]).limit(10)
mx_record_count_top_ten_df = mx_record_checked_count.orderBy(['count'], ascending = [False]).limit(10)

Finally, the columns `a_record` and `mx_record` are removed from the `domains_checked_df` data frame. Furthermore, the first 15 rows of the data frame are displayed as a check for writing to the database:

In [ ]:
# Remove columns and show the data frame
domains_checked_df = domains_checked_df.drop("a_record").drop("mx_record")
domains_checked_df.limit(15).toPandas()

In [ ]:
# Display the data frame
a_record_count_top_ten_df.limit(10).toPandas()

In [ ]:
# Display the data frame
mx_record_count_top_ten_df.limit(10).toPandas()

After the generated data frames have been checked, they must be written to the PostgreSQL database for data visualisation. To enable a faster write speed, `repartition` and `batchsize` are specified:

In [ ]:
# Write data frames to the PostgreSQL database
a_record_count_top_ten_df.write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "a_record_checked_count_global") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

mx_record_count_top_ten_df.write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "mx_record_checked_count_global") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

domains_checked_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_records_checked") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

## 2. Determination of redirects and HTTP status codes

This chapter contains the definition of the `redirects` and `HTTP status` codes to the top level domains. In the following, the functions are imported from the function notebook `Functions.ipynb` in order to use them for the definition of the `UDFs`:

In [ ]:
# Import from Funtions.ipynb
from ipynb.fs.full.Functions import getRedirectUrl, getStatusCodeUrl

# Creating of UDF's
udf_getRedirectUrl = udf(getRedirectUrl, StringType())
udf_getStatusCodeUrl = udf(getStatusCodeUrl, IntegerType())

In this block, the `redirections` and the `HTTP status codes` are determined by calling the UDF functions. For this purpose, the column `top_level_domain` is passed as a parameter. Subsequently, the redirections and status codes are stored in new columns:

In [ ]:
# Create the new columns with the results
domains_redirect_df = domains_df.withColumn("redirection", udf_getRedirectUrl("top_level_domain")) \
                                .withColumn("status_code", udf_getStatusCodeUrl("top_level_domain"))

For storage in the PostgreSQL database, the `a_record` and `mx_record` columns are removed from the `domains_redirect_df` data frame and the first 15 rows of the data frame are displayed for checking for writing to the database:

In [ ]:
# Remove columns and show the data frame
domains_redirect_df = domains_redirect_df.drop("a_record").drop("mx_record")
domains_redirect_df.limit(15).toPandas()

In [ ]:
domains_redirect_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_redirection") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

## 3. MX record location and ASN determination

In this section, the `locations` and `providers` are identified based on the ANS using the MX records already checked. For each MX record the iso code, location, postal code, latitude, longitude and the organisation of the autonomous system are presented:

In [ ]:
# Read data from the database
domain_records_checked_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", "domain_records_checked") \
                .option("user", user) \
                .option("password", password) \
                .load()

# Drop columns
domain_records_checked_df = domain_records_checked_df.drop("a_record_checked").drop("a_record_checked_error").drop("mx_record_checked_error")

# Display the data frame
domain_records_checked_df.limit(20).toPandas()

Next, the functions from the function notebook `Functions.ipynb` that are to be used for the definition of the `UDFs` are imported. After that, StructTypes of StructFields are created to define the return types:

In [ ]:
# Import functions from Funtions.ipynb
from ipynb.fs.full.Functions import getARecords, getGeoLite2_Location, getGeoLite2_ASN

schema_location = StructType([
    StructField("iso_code", StringType(), True),
    StructField("city", StringType(), True),
    StructField("postal", StringType(), True),
    StructField("latitude", StringType(), True),
    StructField("longitude", StringType(), True)])

schema_asn = StructType([
    StructField("autonomous_system_organization", StringType(), True)])

udf_getARecords = udf(getARecords, ArrayType(StringType()))
udf_getGeoLite2_Location = udf(getGeoLite2_Location, schema_location)
udf_getGeoLite2_ASN = udf(getGeoLite2_ASN, schema_asn)


In order to be able to perform the location and provider query, the ip address of the mx records must first be found out. Based on this, the `locations` and the `providers` are determined and the data frame `domains_mx_record_geolite2_df` is generated:

In [ ]:
# Create the new columns with the results
domains_mx_record_geolite2_df = domain_records_checked_df.select(domain_records_checked_df.top_level_domain,explode(domain_records_checked_df.mx_record_checked).alias('mx_record_checked'))
domains_mx_record_geolite2_df = domains_mx_record_geolite2_df.withColumn("mx_record_ip", udf_getARecords("mx_record_checked")) \
                            .withColumn('mx_record_ip', explode(col('mx_record_ip'))) \
                            .withColumn("location", udf_getGeoLite2_Location("mx_record_ip")) \
                            .withColumn("asn", udf_getGeoLite2_ASN("mx_record_ip")) \
                            .select("top_level_domain", "mx_record_checked", "mx_record_ip", "location.*", "asn.*")

For storage in the PostgreSQL database, the first 15 rows of the data frame are displayed for checking for writing to the database:

In [ ]:
# Display the data frame
domains_mx_record_geolite2_df.limit(15).toPandas()

In [ ]:
# Write the data frame to the PostgreSQL database
domains_mx_record_geolite2_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_mx_record_geolite2") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

## 4. IPv6 and SOA information fetch

This section contains the fetch of IPv6, nameserver information and a ranking of the top ten master nameservers and the companies behind.
The following block generates the required functions as user defined function.

In [ ]:
# Import functions from Funtions.ipynb
from ipynb.fs.full.Functions import IPv6Record, IPv6Record_error, getSOAInformation, getSOAInformation_error, getNameServers, getNameServers_error

udf_IPv6Record = udf(IPv6Record, BooleanType())
udf_IPv6Record_error = udf(IPv6Record_error, IntegerType())
udf_getSOAInformation = udf(getSOAInformation, ArrayType(StringType()))
udf_getSOAInformation_error = udf(getSOAInformation_error, IntegerType())
udf_getNameServers = udf(getNameServers, ArrayType(StringType()))
udf_getNameServers_error = udf(getNameServers_error, IntegerType())

### IPv6 information
The IPv6 information provided by the following code are the availability of IPv6 (`ipv6_available`, boolean) and the possible error code during a request (`ipv6_error`). Therefore, the dataframe 'domains_ipv6_df' is created.

In [ ]:
# Create df for ipv6 data
domains_ipv6_df = domains_df.withColumn("ipv6_available", udf_IPv6Record('top_level_domain'))\
                        .withColumn("ipv6_error", udf_IPv6Record_error("top_level_domain"))\
                        .drop('mx_record').drop('a_record')

domains_ipv6_df.limit(10).toPandas()

In [ ]:
#Write the data frame to the PostgreSQL database
domains_ipv6_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "ip_v6_information") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

### SOA information
Among the SOA record, the names of the nameservers `nameservers` (iincluding master server) are fetched in the following. The variable `nameservers_error` indicates if problems ocurred during the request wheres as `nameservers_count` contains the number of nameservers used per domain.

In [ ]:
# Create df for SOA data and drop unnecessary columns
domains_soa_df = domains_df.withColumn("soa_information", udf_getSOAInformation("top_level_domain"))\
                        .withColumn("soa_information_error", udf_getSOAInformation_error("top_level_domain"))\
                        .withColumn("nameservers", udf_getNameServers("top_level_domain"))\
                        .withColumn("nameservers_error", udf_getNameServers_error("top_level_domain"))\
                        .drop('mx_record').drop('a_record')

# Add nameserver count variable
def count_arr(arr): return 0 if arr == None else len(arr)
count_arr_udf = udf(count_arr, IntegerType())
domains_soa_df = domains_soa_df.withColumn("nameservers_count", count_arr_udf('nameservers'))

The details concerning the master nameserver are provided as ArrayType. As this is inconvenient with regard to the data structure (atomicity), the column `soa_infos_rep` is converted into a string type to separate its contents into separate variables.
These are the name of the primary master nameserver `soa_name`, the refresh time of the SOA record executed by the secondary nameservers (`refresh`), the 'Time to live' (TTL), which defines how long a secondary nameserver caches the requested SOA information `minimum`.

In [ ]:
# Change ArrayType<String> into String as preparation for information separation
domains_soa_df = domains_soa_df.withColumn("soa_infos_rep", concat_ws(" ", "soa_information"))

# Split SOA information into separate columns (all String)
split_col = split(domains_soa_df['soa_infos_rep'], ' ')
domains_soa_df = domains_soa_df.withColumn('soa_name', split_col.getItem(0))\
                        .withColumn('refresh', split_col.getItem(3))\
                        .withColumn('minimum', split_col.getItem(6))

# Helping function to catch empty entries concerning the master nameserver
def replace_empty_strings(x):
    return when(col(x) == "", None).otherwise(col(x))

domains_soa_df = domains_soa_df.withColumn("soa_name", replace_empty_strings("soa_name"))

Some fetched entries contain unnecessary characters which need to be removed (dot at the end of the name).

In [ ]:
# Remove last dot per soa mname
domains_soa_df = domains_soa_df.withColumn('soa_name', regexp_replace('soa_name', '.$', ''))   

# Remove last dot per nameserver entry
lambda_dot_remove = lambda arr: [x[:-1] for x in arr]
def fn_remove_dot(arr): return None if arr == None else lambda_dot_remove(arr)
udf_remove_last_char_in_array = udf(fn_remove_dot, ArrayType(StringType()))

domains_soa_df = domains_soa_df \
    .select("*") \
    .withColumn('nameservers', udf_remove_last_char_in_array(col('nameservers')))

In [ ]:
# Change data type of the time setting columns into int
domains_soa_df = domains_soa_df.withColumn("refresh", domains_soa_df["refresh"].cast(IntegerType()))\
                        .withColumn("minimum", domains_soa_df["minimum"].cast(IntegerType()))\
                        .drop('soa_information').drop('soa_infos_rep')

In [ ]:
#Write the data frame to the PostgreSQL database
domains_soa_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "soa") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

### Top ten master namservers and related company information
This section creates a dataframe concerning the top ten master neameservers and the companies behind.

In [ ]:
# Count the occrence of SOA records
soa_name_count_top_ten_df = domains_soa_df.withColumn('soa_name', (col('soa_name'))) \
        .groupBy('soa_name') \
        .count()

soa_name_count_top_ten_df = soa_name_count_top_ten_df.orderBy(['count'], ascending = [False]).limit(10)
soa_name_count_top_ten_df.limit(5).toPandas()

The company informations per master nameserver will be added by a request of its IP adresses `ipv4`. The IP address(es) are the used within the functions concerning GeoLite2 to get the company information ()

In [ ]:
# Remove None entries (copmany information not available)
soa_name_count_top_ten_df = soa_name_count_top_ten_df.na.drop(subset=["soa_name"])

soa_name_count_top_ten_df = soa_name_count_top_ten_df.withColumn("ipv4", udf_getARecords("soa_name"))\
                      .withColumn('ipv4', explode(col('ipv4'))) \
                      .withColumn("location", udf_getGeoLite2_Location("ipv4")) \
                      .withColumn("asn", udf_getGeoLite2_ASN("ipv4")) \
                      .select("soa_name", "count", "ipv4", "location.*", "asn.*")
#soa_name_count_top_ten_df.limit(5).toPandas()

In [ ]:
#Write the data frame to the PostgreSQL database
soa_name_count_top_ten_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "soa_top_ten") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()